# InstaRecipe

Download a full recipe from an Instagram video

**MVP:**

- Paste InstaVideo
- Downloads
- Transcribe audio
- Get frames
- GPT create's recipe
- Pay to use

**v.Next**

- YouTube Video
- FoodNetwork videos?

## Download Insta

In [9]:
!pip3 install instaloader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for instaloader: filename=instaloader-4.13.1-py3-none-any.whl size=67214 sha256=ea1a7bcd0a524d240d56efc68b31e04926574e161dbf8a552a2649d75c2beadd
  Stored in directory: /Users/david.lozzi/Library/Caches/pip/wheels/5d/86/0b/54cbf6960a21dc3873604aa17c8fac43e3a3b677409167b5b5
Successfully built instaloader

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [60]:
import re
import subprocess
import os

url = "https://www.instagram.com/reel/C-Ej9jQts0f/?utm_source=ig_web_copy_link&igsh=MzRlODBiNWFlZA=="
# Parse the value from the URL
video_id = re.search(r'reel/([^/?]+)', url).group(1)

# Run the cli script instaloader
subprocess.run(['instaloader', "--", f"-{video_id}"], cwd='.')

# Rename the folder, the above appa ppends a dash
old_folder_path = os.path.join('.', f"-{video_id}")
new_folder_path = os.path.join('.', video_id)
os.rename(old_folder_path, new_folder_path)


video_dir = os.path.join('.', video_id)
mp4_file = None
if os.path.exists(video_dir) and os.path.isdir(video_dir):
    for root, dirs, files in os.walk(video_dir):
        for file in files:
            if file.endswith('.mp4'):
                mp4_file = os.path.join(root, file)
                break
        if mp4_file:
            break
else:
    print(f"Directory '{video_dir}' does not exist.")

# Print the path of the .mp4 file
print(mp4_file)
folder_path = os.path.dirname(mp4_file)

-C-Ej9jQts0f/2024-07-31_03-03-19_UTC.jpg [crispy sesame salmon bites 🔥 …] -C-Ej9jQts0f/2024-07-31_03-03-19_UTC.mp4 json 
./C-Ej9jQts0f/2024-07-31_03-03-19_UTC.mp4


## Get images from video

In [61]:
import cv2
import tempfile
from skimage.metrics import structural_similarity as ssim
import os

def is_frame_unique_ssim(current_frame, previous_frame, frame_index):
    if previous_frame is None:
        return True

    # Convert the images to grayscale
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

    # Compute SSIM between two images
    ssim_value = ssim(current_frame_gray, previous_frame_gray)

    is_unique = ssim_value < 0.8


    return is_unique
video_fps = 0
def extract_unique_frames(file_path):
    global video_fps
    print("Extracting unique frames from video...")

    with open(file_path, 'rb') as uploaded_file:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp:
            file_bytes = uploaded_file.read()
            temp.write(file_bytes)
            temp_file_name = temp.name

    # Open the temporary file with cv2.VideoCapture
    cap = cv2.VideoCapture(temp_file_name)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total number of frames
    video_fps = cap.get(cv2.CAP_PROP_FPS)

    previous_frame = None
    frame_index = 0
    prev_unique_frame = False
    images = []

    while True:
        print(f"Processing frame {frame_index}...")

        ret, frame = cap.read()
        frame_index += 1

        if not ret:
            break

        if frame_index % 5 != 0:
            continue

        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)

        if is_frame_unique_ssim(frame, previous_frame, frame_index):
            print(f"Frame {frame_index} is unique!")
            prev_unique_frame = True
        else:
            print(f"Frame {frame_index} is not unique!")
            if prev_unique_frame:
                print(f"Frame {frame_index} is the last unique frame!")
                prev_unique_frame = False
                frame_path = os.path.join('.', folder_path, f"frame_{frame_index}.jpg")
                cv2.imwrite(frame_path, frame)
                images.append(frame_path)

        previous_frame = frame.copy()

    cap.release()
    return images

extract_unique_frames(mp4_file)

Extracting unique frames from video...
Processing frame 0...
Processing frame 1...
Processing frame 2...
Processing frame 3...
Processing frame 4...
Frame 5 is unique!
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Frame 10 is not unique!
Frame 10 is the last unique frame!
Processing frame 10...
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Frame 15 is unique!
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Frame 20 is not unique!
Frame 20 is the last unique frame!
Processing frame 20...
Processing frame 21...
Processing frame 22...
Processing frame 23...
Processing frame 24...
Frame 25 is unique!
Processing frame 25...
Processing frame 26...
Processing frame 27...
Processing frame 28...
Processing frame 29...
Frame 30 is not unique!
Frame 30 is the last unique frame!
Processing frame 30...
Processing frame 31...

['././C-Ej9jQts0f/frame_10.jpg',
 '././C-Ej9jQts0f/frame_20.jpg',
 '././C-Ej9jQts0f/frame_30.jpg',
 '././C-Ej9jQts0f/frame_70.jpg',
 '././C-Ej9jQts0f/frame_85.jpg',
 '././C-Ej9jQts0f/frame_125.jpg',
 '././C-Ej9jQts0f/frame_135.jpg',
 '././C-Ej9jQts0f/frame_195.jpg',
 '././C-Ej9jQts0f/frame_215.jpg',
 '././C-Ej9jQts0f/frame_325.jpg',
 '././C-Ej9jQts0f/frame_345.jpg',
 '././C-Ej9jQts0f/frame_360.jpg',
 '././C-Ej9jQts0f/frame_405.jpg']

## Get transcript from video

In [ ]:
import boto3
import time
import os
import load_dotenv

load_dotenv.load_dotenv()

# os.environ['AWS_ACCESS_KEY_ID'] =
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_access_key'
# os.environ['AWS_SESSION_TOKEN'] = 'your_session_token'  # Optional, only if using temporary credentials


# Initialize the S3 and transcribe clients
s3 = boto3.client('s3', region_name='us-west-2')
transcribe = boto3.client('transcribe', region_name='us-west-2')

# Define the video file path and S3 bucket details
video_file_path = mp4_file
bucket_name = 'lozzi-audio'
video_id = 'C-YZAWzhJL-'  # Ensure this matches your video ID
s3_key = f"{video_id}.mp4"

# Upload the video file to S3
s3.upload_file(video_file_path, bucket_name, s3_key)
print(f"Uploaded {video_file_path} to s3://{bucket_name}/{s3_key}")

# Start transcription job
job_name = f"recipe-{video_id}"
job_uri = f"s3://{bucket_name}/{s3_key}"

transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat='mp4',
    LanguageCode='en-US',
    OutputBucketName='your_output_bucket'
)

# Wait for the job to complete
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Waiting for transcription to complete...")
    time.sleep(15)

# Get the transcription result
if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    transcript_uri = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
    print(f"Transcript available at: {transcript_uri}")

In [ ]:
import boto3
from dotenv import load_dotenv
import os

load_dotenv()

os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
print(os.getenv('AWS_ACCESS_KEY_ID'))
print(os.getenv('AWS_SECRET_ACCESS_KEY'))
# Initialize the STS client
sts = boto3.client('sts')

# Get the caller identity
caller_identity = sts.get_caller_identity()

# Print the AWS account ID
account_id = caller_identity['Account']
print(f"Using AWS account ID: {account_id}")

## Group images and script

In [43]:
import re

def get_frame_timestamp(frame_index, video_fps):
    return frame_index / video_fps


def convert_timestamp_to_seconds(timestamp):
    # Split the timestamp into hours, minutes, seconds, and milliseconds
    hours, minutes, seconds_milliseconds = timestamp.split(':')
    seconds, milliseconds = seconds_milliseconds.split(',')

    # Convert each part to seconds
    total_seconds = (int(hours) * 3600) + (int(minutes) * 60) + int(seconds) + (int(milliseconds) / 1000)

    return total_seconds

def read_srt_file(file_path):
  with open(file_path, 'r') as file:
    lines = file.readlines()

  subtitles = []
  for i in range(0, len(lines)):
    line = lines[i]
    if "-->" in line:
        start_time, end_time = re.findall(r'\d{2}:\d{2}:\d{2},\d{3}', line)
        text = lines[i+1].strip()
        subtitle = {
          'start_time': start_time,
          'end_time': end_time,
          'start_seconds': convert_timestamp_to_seconds(start_time),
          'end_seconds': convert_timestamp_to_seconds(end_time),
          'text': text
        }
        subtitles.append(subtitle)

  return subtitles

srt_file = os.path.join(folder_path, 'recipe1.srt')
subtitles = read_srt_file(srt_file)
# display(subtitles)

frames = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.jpg'):
        # Extract the number from the file name
        match = re.search(r'\d+', file_name)
        if match:
            number = match.group()
            frames.append({"file_name": file_name, "number": int(number), "time": get_frame_timestamp(int(number), video_fps)})

# display(frames)

frames.sort(key=lambda x: x['number'])

for subtitle in subtitles:
    start = subtitle['start_seconds']
    end = subtitle['end_seconds']
    text = subtitle['text']
    subtitle['frames'] = []

    for frame in frames:
        if frame['time'] >= start and frame['time'] <= end:
            subtitle['frames'].append(frame)

display(subtitles)

[{'start_time': '00:00:00,009',
  'end_time': '00:00:02,789',
  'start_seconds': 0.009,
  'end_seconds': 2.789,
  'text': "I cannot wait any longer. I'm so ready for soup season.",
  'frames': [{'file_name': 'frame_10.jpg',
    'number': 10,
    'time': 0.3333333333333333},
   {'file_name': 'frame_40.jpg', 'number': 40, 'time': 1.3333333333333333},
   {'file_name': 'frame_70.jpg', 'number': 70, 'time': 2.3333333333333335}]},
 {'start_time': '00:00:02,799',
  'end_time': '00:00:04,179',
  'start_seconds': 2.799,
  'end_seconds': 4.179,
  'text': 'This Mexican street corn,',
  'frames': [{'file_name': 'frame_85.jpg',
    'number': 85,
    'time': 2.8333333333333335},
   {'file_name': 'frame_95.jpg', 'number': 95, 'time': 3.1666666666666665},
   {'file_name': 'frame_110.jpg', 'number': 110, 'time': 3.6666666666666665},
   {'file_name': 'frame_125.jpg', 'number': 125, 'time': 4.166666666666667}]},
 {'start_time': '00:00:04,190',
  'end_time': '00:00:07,650',
  'start_seconds': 4.19,
  'end

## Generative

In [57]:
import base64
import httpx
import json
import load_dotenv

load_dotenv.load_dotenv()

def system_message(content):
    return {"role": "system", "content": content}


def user_message(content):
    return {"role": "user", "content": content}

def image_message(text, base64_images):
    images = []
    for base64_image in base64_images:
        images.append({"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}})

    return {
        "role": "user",
        "content": [
            {"type": "text", "text": text},
            *images
        ],
    }

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string

social = ""
with open(f"{folder_path}/2024-08-07_19-52-36_UTC.txt", "r") as f:
    social = f.read()

async def call_gpt():
    prompt = """You are a GPT, a helpful recipe creator.

# Your Goal
You will be provided text and images from an Instagram video. Your task is to generate a recipe based on the text and images provided.

# Your Tasks
1. The first user message is the creator's comments about their video. It may contain the recipe as well. If so, use their recipe as a reference for your recipe.
2. Each piece of text is from the video audio, and the important frames from the video during that time.
3. Review the text in relation to the images to understand what is going on at this part of the video. Pay attention to ingredients, \
cooking techniques, and any other relevant information.
4. Use this information to generate a recipe as close to the video as possible. Use your knowledge of cooking to ensure the recipe makes sense.
5. Include number of servings, total time, ingredients, step-by-step instructions, and nutrional information in your recipe.
"""

    messages = [system_message(prompt), user_message(f"SOCIAL\nThe following are the creator's comments about their video:\n\n{social}")]
    for subtitle in subtitles:
        text = subtitle['text']
        base64_images = []
        for frame in subtitle['frames']:
            base64_images.append(image_to_base64(os.path.join(folder_path, frame['file_name'])))
        messages.append(image_message(text, base64_images))

    # display(messages)

    print(f"starting generative with {len(messages)} messages")
    payload_json = {
        "messages": messages,
        "model": "gpt-4o",
        "temperature": 0.8,
        "top_p": 0.95,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": 800,
        "stream": False,
    }
    payload = json.dumps(payload_json)

    headers = {
        "Authorization": f'Bearer {os.getenv("OPENAI_API_KEY")}',
        "Content-Type": "application/json",
    }
    url = "https://api.openai.com/v1/chat/completions"
    async with httpx.AsyncClient(verify=False) as client:
        response = await client.post(url, headers=headers, data=payload, timeout=120)
        if response.status_code != 200:
            print(
                f"ERROR calling generative API: {response.status_code} {response.text}"
            )
        else:
            try:
                json_resp = response.json()
                with open(f"generative_usage.json", "a") as f:
                    f.write(f'{json.dumps(json_resp.get("usage", {}))}\n')

                print(json_resp.get("choices")[0].get("message").get("content"))

            except Exception as e:
                print(f"ERROR parsing generative response: {e}")
                display(response)

await call_gpt()

starting generative with 10 messages
Here's a detailed recipe for the Mexican Street Corn White Chicken Chili:

### Mexican Street Corn White Chicken Chili

#### Servings: 6
#### Total Time: 1 hour

#### Ingredients:

- 4 boneless, skinless chicken breasts
- 1 yellow onion, chopped
- 1 jalapeno, diced
- 4 cups chicken bone broth
- 1.5 cups sour cream
- 1/2 cup shredded monterey jack cheese (or a blend of monterey jack and mild cheddar)
- 4 cloves garlic, minced
- 1/2 tablespoon dried oregano
- 1/2 teaspoon chili powder
- 2 cups frozen sweet white corn
- 1/2 cup fresh cilantro, chopped
- Juice of 1 lime
- 3 tablespoons cornstarch
- 3 tablespoons water
- Olive oil
- Cotija cheese (for topping)
- Bacon crumbles (for topping)
- Tortilla strips (for topping)
- Sliced avocado (for topping)
- Lime wedges (for serving)
- Salt and pepper to taste

#### Directions:

1. **Prepare the Base:**
   - In a large soup pot, heat 1 tablespoon of olive oil over medium heat. 
   - Add chopped onion and dic